In [1]:
import json

from glob import glob
from unidiff import PatchSet

In [2]:
!git clone git@github.com:CognitionAI/devin-swebench-results.git

Cloning into 'devin-swebench-results'...
remote: Enumerating objects: 582, done.
remote: Counting objects: 100% (582/582), done.
remote: Compressing objects: 100% (570/570), done.
remote: Total 582 (delta 12), reused 579 (delta 9), pack-reused 0
Receiving objects: 100% (582/582), 571.31 KiB | 6.35 MiB/s, done.
Resolving deltas: 100% (12/12), done.
Updating files: 100% (580/580), done.


In [7]:
def convert_devin_txt_to_pred(pred_file):
    inst_id = pred_file.split("/")[-1].split("-diff")[0]
    pred = open(pred_file).read()
    try:
        PatchSet(pred)
    except:
        print(f"{inst_id}: Prediction patch is malformed")
    return {
        "model_name_or_path": "devin-20240406",
        "instance_id": inst_id,
        "model_patch": pred
    }

In [8]:
predictions = []
for pred_file in \
    glob("devin-swebench-results/output_diffs/fail/*.txt") + \
    glob("devin-swebench-results/output_diffs/pass/*.txt"):
    predictions.append(convert_devin_txt_to_pred(pred_file))
len(predictions)

570

In [9]:
predictions[0]

{'model_name_or_path': 'devin-20240406',
 'instance_id': 'django__django-16745',
 'model_patch': 'diff --git a/django/core/validators.py b/django/core/validators.py\nindex 6c622f5788..7a1aff3fe5 100644\n--- a/django/core/validators.py\n+++ b/django/core/validators.py\n@@ -397,8 +397,9 @@ class StepValueValidator(BaseValidator):\n     message = _("Ensure this value is a multiple of step size %(limit_value)s.")\n     code = "step_size"\n \n-    def compare(self, a, b):\n-        return not math.isclose(math.remainder(a, b), 0, abs_tol=1e-9)\n+    def compare(self, a, b, min_value=0):\n+        offset = a - min_value\n+        return not math.isclose(math.remainder(offset, b), 0, abs_tol=1e-9)\n \n \n @deconstructible\n'}

In [10]:
with open("devin_predictions.jsonl", "w") as f:
    for pred in predictions:
        print(json.dumps(pred), file=f, flush=True)